### Name of the Student: Nicholas Furi

### Predict winner soccer games: Acquisition of the data 

In the following notebook, the code of data acquisition and elaboration is provided.
The type of data that is acquired referce to the Italian Soccer League (Serie A).
The main goal of this analysis is to build models that are able to predict the winner of each game of the league. 
- The data acquired is of tree different types:
    - Soccer Stats, data that cointain information about the match of each game. 
    - Betting Odds, which are the odds of each match provided from main betting sites 
    - Sentiment Analysis data, which provide insights of the opinions and mood of the fan base and of the pubblic opinion scraped from twitter
    
- The first two type od data (Soccer Stats and Bettings Odds) are taken from the following site, https://www.football-data.co.uk/italym.php. Their time range goes from 2015 to the end of 2022.
- The third data type (Sentiment Analysis) is scraped from twitter

In [15]:
import pandas as pd
from functools import reduce
import numpy as np


In [16]:
# This are all the 8 csv file downloaded from the site provided above

Seriea2223 = pd.read_csv('CSV Campionati/Serie A 2223.csv')
Seriea2122 = pd.read_csv('CSV Campionati/Serie A 2122.csv')
Seriea2021 = pd.read_csv('CSV Campionati/Serie A 2021.csv')
Seriea1920 = pd.read_csv('CSV Campionati/Serie A 1920.csv')
Seriea1819 = pd.read_csv('CSV Campionati/Serie A 1819.csv')
Seriea1718 = pd.read_csv('CSV Campionati/Serie A 1718.csv')
Seriea1617 = pd.read_csv('CSV Campionati/Serie A 1617.csv')
Seriea1516 = pd.read_csv('CSV Campionati/Serie A 1516.csv')

frames = [Seriea2223,Seriea2122,Seriea2021,Seriea1920,Seriea1819,Seriea1718,Seriea1617,Seriea1516]
df_main = pd.concat(frames)

In [17]:
df_main

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA
0,I1,13/08/2022,17:30,Milan,Udinese,4.0,2.0,H,2.0,2.0,D,14.0,9.0,5.0,4.0,16.0,13.0,5.0,1.0,1.0,4.0,0.0,0.0,1.44,4.75,7.00,1.41,4.75,7.50,1.4,4.80,8.5,1.43,4.66,8.84,1.38,4.60,8.50,1.36,4.60,9.00,1.46,5.10,9.00,1.41,4.75,8.14,1.72,2.10,1.77,2.16,1.82,2.21,1.73,2.14,-1.25,1.94,1.96,1.98,1.94,1.98,2.01,1.92,1.94,1.44,4.75,6.50,1.47,4.5,6.50,1.45,4.5,7.25,1.47,4.94,7.12,1.47,4.33,7.00,1.45,4.6,5.75,1.54,5.05,7.50,1.48,4.58,6.71,1.66,2.20,1.74,2.21,1.82,2.28,1.72,2.16,-1.25,1.98,1.92,2.02,1.91,2.17,1.94,2.06,1.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I1,13/08/2022,17:30,Sampdoria,Atalanta,0.0,2.0,A,0.0,1.0,A,11.0,9.0,1.0,3.0,13.0,21.0,3.0,2.0,4.0,6.0,0.0,0.0,4.33,3.90,1.75,4.33,3.80,1.77,4.3,3.90,1.8,4.72,3.98,1.78,4.50,3.80,1.73,4.75,3.70,1.73,4.75,4.12,1.84,4.45,3.91,1.77,1.72,2.10,1.73,2.22,1.76,2.26,1.69,2.18,0.75,1.90,2.00,1.93,1.99,1.93,2.02,1.88,1.97,5.25,4.00,1.61,4.80,3.9,1.68,4.80,3.8,1.73,5.60,4.13,1.66,5.50,3.80,1.62,5.75,4.0,1.57,5.75,4.30,1.73,5.29,4.06,1.64,1.66,2.20,1.69,2.29,1.73,2.35,1.67,2.24,0.75,2.07,1.83,2.11,1.82,2.13,1.87,2.06,1.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I1,13/08/2022,19:45,Lecce,Inter,1.0,2.0,A,0.0,1.0,A,10.0,19.0,4.0,9.0,21.0,19.0,6.0,8.0,4.0,2.0,0.0,0.0,10.00,5.75,1.28,9.75,5.50,1.30,9.5,5.75,1.3,10.75,5.92,1.31,11.00,5.50,1.27,12.00,5.50,1.25,12.00,6.15,1.33,10.22,5.67,1.30,1.57,2.37,1.61,2.46,1.65,2.46,1.60,2.37,1.50,2.01,1.89,2.07,1.86,2.07,1.92,2.00,1.87,8.00,5.25,1.36,9.75,5.5,1.30,7.75,5.5,1.37,8.66,5.55,1.37,9.00,5.00,1.33,10.00,5.5,1.30,10.00,5.70,1.38,8.53,5.41,1.35,1.53,2.50,1.55,2.60,1.58,2.70,1.52,2.55,1.50,1.90,2.00,1.91,2.02,1.96,2.06,1.89,1.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I1,13/08/2022,19:45,Monza,Torino,1.0,2.0,A,0.0,1.0,A,15.0,13.0,4.0,6.0,6.0,17.0,2.0,7.0,1.0,2.0,0.0,0.0,2.60,3.30,2.75,2.55,3.25,2.80,2.6,3.30,2.8,2.64,3.36,2.90,2.50,3.25,2.80,2.55,3.10,2.88,2.68,3.48,2.94,2.58,3.29,2.84,2.10,1.72,2.09,1.83,2.21,1.84,2.09,1.76,0.00,1.84,2.06,1.87,2.06,1.87,2.08,1.84,2.04,2.55,3.10,2.90,2.60,3.2,2.75,2.55,3.2,2.85,2.56,3.30,3.05,2.55,3.10,2.88,2.55,3.1,2.90,2.81,3.38,3.05,2.57,3.22,2.90,2.10,1.72,2.10,1.81,2.19,1.83,2.10,1.75,0.00,1.79,2.11,1.80,2.15,1.88,2.16,1.83,2.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,I1,14/08/2022,17:30,Fiorentina,Cremonese,3.0,2.0,H,2.0,1.0,H,27.0,13.0,11.0,4.0,10.0,19.0,11.0,4.0,0.0,3.0,0.0,1.0,1.36,5.00,8.50,1.39,4.80,7.75,1.4,5.00,7.5,1.39,5.33,8.49,1.36,4.60,9.00,1.33,4.80,9.50,1.42,5.40,9.50,1.38,5.02,8.16,1.66,2.20,1.63,2.40,1.70,2.40,1.63,2.25,-1.25,1.85,2.05,1.85,2.08,1.88,2.08,1.85,2.02,1.60,4.20,5.25,1.58,4.1,5.50,1.63,4.2,5.25,1.62,4.35,5.64,1.57,4.00,5.80,1.57,4.1,5.75,1.64,4.75,6.50,1.59,4.22,5.62,1.80,2.00,1.82,2.09,1.82,2.20,1.77,2.09,-1.00,2.02,1.88,2.06,1.87,2.07,1.97,1.99,1.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [25]:
df_main.shape

(2811, 126)

In [22]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2811 entries, 0 to 380
Columns: 126 entries, Div to LBA
dtypes: float64(119), object(7)
memory usage: 2.7+ MB


In [19]:
df_main.isnull().sum()

Div            1
Date           1
Time        1521
HomeTeam       1
AwayTeam       1
            ... 
BbMxAHA     1291
BbAvAHA     1291
LBH         1671
LBD         1671
LBA         1671
Length: 126, dtype: int64

#### The concatenate Dataframe seem to have several Nan due to the fact that not all of the csv file have the same number of columns.

#### In the following code I will check which are the columns in common between all of the csv file


In [13]:
a = list(reduce(set.intersection, map(set, [Seriea2122,Seriea2021,Seriea2223,
                                        Seriea1920,Seriea1819,Seriea1718,Seriea1617,Seriea1516])))
a

['IWH',
 'FTR',
 'Date',
 'WHD',
 'HF',
 'HC',
 'FTHG',
 'HST',
 'HR',
 'AC',
 'HS',
 'HTAG',
 'AR',
 'VCD',
 'AY',
 'B365A',
 'Div',
 'PSCA',
 'AS',
 'IWD',
 'AF',
 'HTHG',
 'B365D',
 'WHH',
 'WHA',
 'BWH',
 'PSH',
 'VCA',
 'AST',
 'VCH',
 'FTAG',
 'PSA',
 'PSCD',
 'AwayTeam',
 'B365H',
 'IWA',
 'BWA',
 'PSCH',
 'HTR',
 'PSD',
 'HY',
 'BWD',
 'HomeTeam']

## Conducting Twitter Scrape in order to get a Subjectivity Score and a Polarity Score

#### This scrape uses a particular tool called snscrape.modules.twitter:
- the scrape is conducted singolary to each season, due to the long time of the operation
- for each game, a maximum of 500 tweets are obtain and the following operation ore conducted:
    - cleaning the tweet (def cleanUpTweet)
    - Identification of the tweet as a Positve/Negative/Neutral for Subjectivity
    - Identification of the tweet as a Positve/Negative/Neutral for Subjectivity
- In order to obtain a score, the following operation have been conducted:
    - Subjectivity_score --> Calculated the sum of each Postivity tweet for the match and divided by the sum of the the Positive tweets and Negative tweets.
    - Polarity_score --> Calculated the sum of each Postivity tweet for the match and divided by the sum of the the Positive tweets and Negative tweets.
    


- The polarity score an the subjectivity score are calculated for the following reason:
    - identify the mood of the fan base
    - Polarity represent the emotions in the tweets
    - Subjectivity represent the opitions express in the tweet

In [3]:
import pandas as pd
import numpy as np
import snscrape.modules.twitter as sntwitter
import re   # regular expretion library
from textblob import TextBlob

In [8]:
# read the first file of the leage in 2015/16
df = pd.read_csv('CSV Campionati/Serie A 1516.csv')
# drop all the Nan Values
df.dropna(inplace=True)

#Selecting the columns needed based on the script run before. This are all the columns that are common
#in all the other file 
df2 = df[['Date','Div','HomeTeam','HS','HST','HC','HF','HY','HR',
            'AwayTeam','AS','AST','AC','AF','AY','AR',
            'HTHG','HTAG','HTR','FTHG','FTAG','FTR',
            'VCA','VCD','VCH','IWA','IWD','IWH',
            'BWA','BWD','BWH','B365A','B365D','B365H',
            'WHH','WHD','WHA']].copy()

### Description of the columns:

- Div, League Division
- AC, Away Team Corners
- AF, Away Team Fouls Committed
- AR, Away Team Red Cards
- AS, Away Team Shots
- AST, Away Team Shots on Target
- Away, Team, Away Team
- AY, Away Team Yellow Cards
- B365A, Bet365 away win odds
- B365D,  Bet365 draw odds
- B365H, Bet365 home win odds
- BWA, Bet&Win away win odds
- BWD, Bet&Win draw odds
- BWH, Bet&Win home win odds
- Date, Match Date (dd/mm/yy)
- FTAG and AG, Full Time Away Team Goals
- FTHG and HG, Full Time Home Team Goals
- FTR and Res, Full Time Result (H
- HC, Home Team Corners
- HF, Home Team Fouls Committed
- HomeTeam, Home Team
- HR, Home Team Red Cards
- HS, Home Team Shots
- HST, Home Team Shots on Target
- HTAG, Half Time Away Team Goals
- HTHG,  Half Time Home Team Goals
- HTR, Half Time Result (H
- HY,  Home Team Yellow Cards
- IWA, Interwetten away win odds
- IWD, Interwetten draw odds
- IWH, Interwetten home win odds
- VCA, VC Bet away win odds
- VCD, VC Bet draw odds
- VCH, VC Bet home win odds
- WHA, William Hill away win odds
- WHD, William Hill draw odds
- WHH, William Hill home win odds

#### Most of this null values are from the of the bettings sites. The goal is to recuperate  most of them by creating the averages of all the Betting odds for AwayOdds, DrawOdds and HomeOdds

In [9]:
# Creating 3 columns with the averages of the Betting odds
# This is done because also so that is possible to avoid repetition of data. The Odds of the betting sites are slighly 
# different but overall are the same.

# first for away odds
df2['Avg_Odds_Away'] = df2[["VCA","IWA","BWA","B365A","WHA"]].mean(axis=1)


#second for draw odds
df2['Avg_Odds_Draw'] = df2[["VCD","IWD","BWD","B365D","WHD"]].mean(axis=1)

# third for home odds

df2['Avg_Odds_Home'] = df2[["VCH","IWH","BWH","B365H","WHH"]].mean(axis=1)

pd.set_option('display.max_columns', None)



,Date,Div,HomeTeam,HS,HST,HC,HF,HY,HR,AwayTeam,AS,AST,AC,AF,AY,AR,HTHG,HTAG,HTR,FTHG,FTAG,FTR,VCA,VCD,VCH,IWA,IWD,IWH,BWA,BWD,BWH,B365A,B365D,B365H,WHH,WHD,WHA,Avg_Odds_Away,Avg_Odds_Draw,Avg_Odds_Home
0,22/08/15,I1,Lazio,21.0,9.0,10.0,18.0,2.0,0.0,Bologna,9.0,4.0,3.0,8.0,1.0,0.0,2.0,1.0,H,2.0,1.0,H,8.00,4.50,1.45,6.10,4.00,1.5,7.0,3.8,1.50,7.00,4.20,1.50,1.50,4.0,7.00,7.020,4.100,1.490
1,22/08/15,I1,Verona,12.0,7.0,3.0,9.0,2.0,0.0,Roma,21.0,8.0,7.0,11.0,1.0,0.0,0.0,0.0,D,1.0,1.0,D,1.75,3.70,5.50,1.75,3.50,4.5,1.7,3.6,5.00,1.67,3.75,5.50,5.00,3.5,1.73,1.720,3.610,5.100
2,23/08/15,I1,Empoli,15.0,5.0,8.0,15.0,1.0,0.0,Chievo,12.0,3.0,5.0,19.0,2.0,0.0,1.0,0.0,H,1.0,3.0,A,4.00,3.13,2.20,3.00,3.20,2.3,3.4,3.1,2.20,3.60,3.25,2.15,2.20,3.1,3.50,3.500,3.156,2.210
3,23/08/15,I1,Fiorentina,13.0,7.0,4.0,18.0,3.0,0.0,Milan,8.0,0.0,5.0,16.0,2.0,1.0,1.0,0.0,H,2.0,0.0,H,3.75,3.40,2.15,2.90,3.30,2.3,3.4,3.1,2.20,3.50,3.40,2.15,2.20,3.1,3.50,3.410,3.260,2.200
4,23/08/15,I1,Frosinone,13.0,3.0,3.0,10.0,2.0,0.0,Torino,26.0,7.0,12.0,11.0,2.0,0.0,1.0,0.0,H,1.0,2.0,A,2.25,3.13,3.75,2.30,3.30,2.9,2.2,3.1,3.40,2.20,3.25,3.50,3.40,3.1,2.25,2.240,3.176,3.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,15/05/16,I1,Empoli,10.0,3.0,3.0,12.0,2.0,0.0,Torino,18.0,5.0,4.0,12.0,1.0,0.0,1.0,0.0,H,2.0,1.0,H,2.70,3.50,2.60,3.00,3.20,2.3,2.7,3.4,2.45,2.70,3.50,2.55,2.38,3.5,2.88,2.796,3.420,2.456
376,15/05/16,I1,Genoa,11.0,2.0,8.0,19.0,3.0,0.0,Atalanta,13.0,4.0,5.0,14.0,3.0,0.0,0.0,0.0,D,1.0,2.0,A,4.10,3.80,1.87,3.80,3.45,1.9,3.9,3.5,1.90,4.20,3.60,1.85,1.95,3.3,4.00,4.000,3.530,1.894
377,15/05/16,I1,Lazio,16.0,6.0,6.0,8.0,0.0,0.0,Fiorentina,16.0,5.0,4.0,10.0,1.0,0.0,1.0,3.0,A,2.0,4.0,A,3.30,3.60,2.20,3.30,3.30,2.1,3.1,3.5,2.15,3.50,3.40,2.10,2.15,3.3,3.40,3.320,3.420,2.140
378,15/05/16,I1,Palermo,5.0,3.0,4.0,18.0,2.0,1.0,Verona,14.0,4.0,4.0,19.0,2.0,1.0,1.0,0.0,H,3.0,2.0,H,12.00,6.25,1.17,10.50,6.50,1.2,11.5,6.5,1.20,13.00,7.00,1.20,1.17,6.5,13.00,12.000,6.550,1.188


### Splitting each row into two

- From the dataFrame above there is are columns for the 'HomeTeam' and the 'AwayTeam'. The idea of splitting each row into two is that the model can train on each team as in input

The following action have been conducted:
- 1.1 Creating a double for each row
- 1.2 For the each first and then every other third column creating a new column called Venue and assigning the value Home, for all the rest of the other rows the value will be Away.
- 1.3 For each row that is identified as 'Away', changing the location of the Away team under the Home team, and done the rest for the stats columns
- 1.4 Rename of each column to names appropriete to the action conducted above
- 1.5 Drop of the columns 'HTR','FTR'. Both represent the end score for the half time of the game (HTF) and for the full time of the game (FTR). Creating new columns that identify the half/final result of the game with the main subject the the team name that is present on the Team column.


In [10]:
# 1.1 Create the column Away and Home
 

df2_split = pd.DataFrame(np.repeat(df2.values, 2, axis=0))
df2_split.columns = df2.columns

# 1.2

df2_split['Venue'] = np.where(df2_split.index%2==0,'Home','Away')



# 1.3 For the away team, it change it location under home team

m = df2_split['Venue'] == 'Away'


df2_split.loc[m,['AwayTeam','HomeTeam']] = (
    df2_split.loc[m, ['HomeTeam','AwayTeam']].values)


#change the value of the stats location away team, under the Home team


df2_split.loc[m,['HS','HST','HC','HF','HY','HR','HTHG','FTHG','AS','AST','AC','AF','AY','AR','HTAG','FTAG']] = (
    df2_split.loc[m, ['AS','AST','AC','AF','AY','AR','HTAG','FTAG','HS','HST','HC','HF','HY','HR','HTHG','FTHG']].values)

# 1.4 change the names of the columns of the dataframe

df2_split.rename({'HomeTeam':'Team','HS':'TS','HST':'TST','HC':'TC','HF':'TF','HY':'TY','AwayTeam':'Opponent','AS':'OS','AST':'OST','AC':'OC','AF':'OF','AY':'OY','AR':'OR','HTHG':'HTTG','HTAG':'HTOG','FTHG':'FTTG','FTAG':'FTOG'},axis=1, inplace=True)   
df2_split

#Drop Two column which will then be recreatated in a loop
df2_split = df2_split.drop(['HTR', 'FTR'], axis=1)

# 1.5 Create the new column HTR and Change the location of the values for the Half time column result

new_HTR = []

for i in range (len(df2_split['HTTG'])):
    HTTG_value = df2_split['HTTG'][i]
    HTOG_value = df2_split['HTOG'][i]
    
    if HTTG_value > HTOG_value:
        new_HTR.append("W")
    elif HTTG_value == HTOG_value:
        new_HTR.append("D")
    else:
        new_HTR.append("L")

df2_split.insert(18, 'new_HTR', new_HTR)


# Create the new column FTR and Change the location of the values for the FULL time column result


new_FTR = []

for i in range (len(df2_split['FTTG'])):
    FTTG_value = df2_split['FTTG'][i]
    FTOG_value = df2_split['FTOG'][i]
    
    if FTTG_value > FTOG_value:
        new_FTR.append("W")
    elif FTTG_value == FTOG_value:
        new_FTR.append("D")
    else:
        new_FTR.append("L")

df2_split.insert(21, 'new_FTR', new_FTR)
df2_split

,Date,Div,Team,TS,TST,TC,TF,TY,HR,Opponent,OS,OST,OC,OF,OY,OR,HTTG,HTOG,new_HTR,FTTG,FTOG,new_FTR,VCA,VCD,VCH,IWA,IWD,IWH,BWA,BWD,BWH,B365A,B365D,B365H,WHH,WHD,WHA,Avg_Odds_Away,Avg_Odds_Draw,Avg_Odds_Home,Venue
0,22/08/15,I1,Lazio,21.0,9.0,10.0,18.0,2.0,0.0,Bologna,9.0,4.0,3.0,8.0,1.0,0.0,2.0,1.0,W,2.0,1.0,W,8.0,4.5,1.45,6.1,4.0,1.5,7.0,3.8,1.5,7.0,4.2,1.5,1.5,4.0,7.0,7.02,4.1,1.49,Home
1,22/08/15,I1,Bologna,9.0,4.0,3.0,8.0,1.0,0.0,Lazio,21.0,9.0,10.0,18.0,2.0,0.0,1.0,2.0,L,1.0,2.0,L,8.0,4.5,1.45,6.1,4.0,1.5,7.0,3.8,1.5,7.0,4.2,1.5,1.5,4.0,7.0,7.02,4.1,1.49,Away
2,22/08/15,I1,Verona,12.0,7.0,3.0,9.0,2.0,0.0,Roma,21.0,8.0,7.0,11.0,1.0,0.0,0.0,0.0,D,1.0,1.0,D,1.75,3.7,5.5,1.75,3.5,4.5,1.7,3.6,5.0,1.67,3.75,5.5,5.0,3.5,1.73,1.72,3.61,5.1,Home
3,22/08/15,I1,Roma,21.0,8.0,7.0,11.0,1.0,0.0,Verona,12.0,7.0,3.0,9.0,2.0,0.0,0.0,0.0,D,1.0,1.0,D,1.75,3.7,5.5,1.75,3.5,4.5,1.7,3.6,5.0,1.67,3.75,5.5,5.0,3.5,1.73,1.72,3.61,5.1,Away
4,23/08/15,I1,Empoli,15.0,5.0,8.0,15.0,1.0,0.0,Chievo,12.0,3.0,5.0,19.0,2.0,0.0,1.0,0.0,W,1.0,3.0,L,4.0,3.13,2.2,3.0,3.2,2.3,3.4,3.1,2.2,3.6,3.25,2.15,2.2,3.1,3.5,3.5,3.156,2.21,Home
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,15/05/16,I1,Fiorentina,16.0,5.0,4.0,10.0,1.0,0.0,Lazio,16.0,6.0,6.0,8.0,0.0,0.0,3.0,1.0,W,4.0,2.0,W,3.3,3.6,2.2,3.3,3.3,2.1,3.1,3.5,2.15,3.5,3.4,2.1,2.15,3.3,3.4,3.32,3.42,2.14,Away
754,15/05/16,I1,Palermo,5.0,3.0,4.0,18.0,2.0,1.0,Verona,14.0,4.0,4.0,19.0,2.0,1.0,1.0,0.0,W,3.0,2.0,W,12.0,6.25,1.17,10.5,6.5,1.2,11.5,6.5,1.2,13.0,7.0,1.2,1.17,6.5,13.0,12.0,6.55,1.188,Home
755,15/05/16,I1,Verona,14.0,4.0,4.0,19.0,2.0,1.0,Palermo,5.0,3.0,4.0,18.0,2.0,1.0,0.0,1.0,L,2.0,3.0,L,12.0,6.25,1.17,10.5,6.5,1.2,11.5,6.5,1.2,13.0,7.0,1.2,1.17,6.5,13.0,12.0,6.55,1.188,Away
756,15/05/16,I1,Udinese,15.0,5.0,6.0,14.0,3.0,1.0,Carpi,10.0,5.0,3.0,17.0,3.0,0.0,0.0,2.0,L,1.0,2.0,L,3.1,3.13,2.2,2.5,3.5,2.5,3.1,3.4,2.2,3.1,3.4,2.3,2.4,3.5,2.8,2.92,3.386,2.32,Home


## Build-in function for:
- 2.1 text cleaning
- 2.2 Subjectivity score
- 2.3 Polarity score
- 2.4 Assignin a category (Negative,Positive,Neutral) to each tweet based a score

In [11]:
# 1.1 The function that the tweets have to go over through

def cleanUpTweet(txt):
        txt = re.sub(r'@[A-Za-z0-9_]+', '',txt)
        txt = re.sub(r'#', '',txt)
        txt = re.sub(r'RT : ', '', txt)
        txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
        return txt
    
# 1.2 first is for the text subjectivity, someone's beliefs feeling or opitions

def getTextSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

# 1.3 function the get the polarity, represents the emotion in a text

def getTextPolarity(txt):
    return TextBlob(txt).sentiment.polarity

# 1.4

def getTextAnalysis(a):
    if a < 0:
        return "Negative"
    elif a==0:
        return "Neutral"
    else:
        return "Positive"

### In order to use the snscrape tool to scrape tweets for twitter, in necessary to create two datetime that scrape tweets before each game happens.
The time laps is of two days before the game. 
So the first datetime create represent the date of 2 days before the game['newdate_-2'], the second datetime is the day before the game['newdate_-1']

In [20]:
import warnings
warnings.filterwarnings('ignore')

# to get the date out have to transform it ot datatime
df2_split['newdate_-1'] = pd.to_datetime(df2_split['Date']).apply(pd.DateOffset(-1))
df2_split['newdate_-2'] = pd.to_datetime(df2_split['Date']).apply(pd.DateOffset(-2))

# Convert the new columns to Date


df2_split['newdate_-2'] = df2_split['newdate_-2'].dt.date
df2_split['newdate_-1'] = df2_split['newdate_-1'].dt.date

df2_split['Date'] = df2_split['Date'].str.replace('/','.')

### Scrapping Tweets, what has been done:

- 3.1 created 2 for loop, the first one loop for each index of the dataframe (each game played), the second loop scrapes 500 tweets for each game. The scrape has 3 features:
    - Hashtage, which is the name of the game. In the code is the first brace
    - Scrape since, is the date from which the scrape starts. In the code in the second brace. df2_split['newdate_-2']
    - Scrape untill, is the date from which the scrape ends. In the code in the third brace. df2_split['newdate_-1']
- 3.2 All of the tweets are appended in a DataFrame that contains the following columns:
    - Username of the tweet
    - Date of the tweet
    - Number of likes
    - Source of tweet
    - String of the tweet
- 3.3 Cleaning the tweets out
- 3.4 Calculate the Subjectivity
- 3.5 Calculate the Polarity
- 3.6 Drop all the columns that how values 0 for subjectivity and Polarity
- 3.7 Apply the text Analysis function to the column both for Subjectivity and Polarity
- 3.8 Calculate the the Percentage of the positive tweets per Polarity
- 3.9 Calculate the the Percentage of the positive tweets per Subjectivity
- 3.10 Saving each scrape for each game in a file 
- 3.11 Append the result of the Polarity, Subjectivity
- 3.12 Create Dataframe that contains polarity_score and subjectivity_score and assigning the names to columns 
- 3.13 Joining togheter the two dataframes, both of stats and betting odds, with the polarity and subjectivity score
- 3.14 Saving the Dataframe inside the folder togheter with all tweets

In [ ]:
polarity_score = []
subjectivity_score = []

#3.1 
for ind in df2_split.index:
    
    attributes_container = []

    for i,tweet in enumerate(sntwitter.TwitterSearchScraper('{} since:{} until:{} '.format(df2_split['Team'][ind],df2_split['newdate_-2'][ind],df2_split['newdate_-1'][ind])).get_items()):
        if i>500:
            break
        #3.2
        attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.sourceLabel, tweet.content])  
        # Creating a dataframe to load the list
        tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Source of Tweet", "Tweet"])
        
        # 3.3
        tweets_df['Tweet'] = tweets_df['Tweet'].apply(cleanUpTweet)
        
        #3.4
        tweets_df['Subjectivity']= tweets_df['Tweet'].apply(getTextSubjectivity)
    
        #3.5
        tweets_df['Polarity']= tweets_df['Tweet'].apply(getTextPolarity)
        
        #3.6
        tweets_df = tweets_df.drop(tweets_df[tweets_df['Tweet']==''].index)
        
        #3.7
        tweets_df["ScorePol"] = tweets_df["Polarity"].apply(getTextAnalysis)
        tweets_df["ScoreSub"] = tweets_df["Subjectivity"].apply(getTextAnalysis)
    
    
        #3.8
        positivePol = tweets_df[tweets_df["ScorePol"] == "Positive"]
        negativePol = tweets_df[tweets_df["ScorePol"] == "Negative"]       
        Pol_score = positivePol.shape[0]/(positivePol.shape[0]+negativePol.shape[0] + 1)
                
    
        #3.9
        positiveSub = tweets_df[tweets_df["ScoreSub"] == "Positive"]
        negativeSub = tweets_df[tweets_df["ScoreSub"] == "Negative"]
        Sub_score = positiveSub.shape[0]/(positiveSub.shape[0]+negativeSub.shape[0] + 1)
        
        
        #3.10
        tweets_df.to_csv("year1516/{}{}tweets_df.csv".format(df2_split['Team'][ind],df2_split['Date'][ind]))
    
        
    #3.11  
    polarity_score.append(Pol_score)
    subjectivity_score.append(Sub_score) 
    print('The polarity Score for roow {} is {}'.format([ind],Pol_score))
    print('The subjectivity Score for roow {} is {}'.format([ind],Sub_score))
    
    
#3.12  
df_score = pd.DataFrame(list(map(list, zip(polarity_score,subjectivity_score))))
df_score.rename({0:'polarity_score',1:'subjectivity_score'},axis=1, inplace=True)


#3.13
df_sa1516 = df2_split.join(df_score)

#3.14
df_sa1516.to_csv("year1516/df_sa1516.csv")

### The scrape done was only for the year 2015/2016 season .
- In order to scrape even for the other seasons in the following csv, such as:
    - Serie A 1617.csv, season 2016/2017
    - Serie A 1718.csv, season 2017/2018
    - Serie A 1819.csv, season 2018/2019
    - Serie A 1920.csv, season 2019/2020
    - Serie A 2021.csv, season 2020/2021
    - Serie A 2122.csv, season 2021/2022
    - Serie A 2223.csv, season 2022/2023
- The following changes have to be done:
    - read a new csv file
    - At point 3.10 of the code, change the name fo the file where the tweets will be stored
    - At point 3.13 change the name of the DataFrame where the combined data (tweets,stats and odds) are saved
    - At point 3.14 change the name of the DataFrame and the name of the file which were setted in the steps above

